## Step 22: 연산자 오버로드 (3)

앞서 우리는 연산자 +와 *에 대해서 오버로드를 진행했다. 이번에는 나머지에 대해서도 구현하도록 하겠다.

1. Function 클래스 상속하여 함수 클래스 구현
2. 파이썬 함수로 사용할 수 있도록 변환
3. Variable 클래스의 연산자 오버로드

### 22.1 음수 (부호 변환)

\_\_neg__(self): 부호 변환 단항 연산자

$$
y = -x
$$
$$
{\delta y \over \delta x} = -1
$$

```python
# 1
class Neg(Function):
    def forward(self, x):
        return -x
    
    def backward(self, gy):
        return -gy

# 2
def neg(x):
    return Neg()(x)

# 3
Variable.__neg__ = neg
```

### 22.2 뺄셈

\_\_sub__(self, other) / \_\_rsub__(self, other): 첫 번째 인수 값에서 두 번째 인수 값을 빼는 이항 연산자

$$
y = x_0 - x_1
$$
$$
{\delta y \over \delta x_0} = 1
$$
$$
{\delta y \over \delta x_1} = -1
$$

```python
class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y
    
    def backward(self, gy):
        return gy, -gy

def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)

def rsub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x1, x0)

Variable.__sub__ = sub
Variable.__rsub__ = rsub
```

### 22.3 나눗셈

\_\_truediv__(self, other) / \_\_rtruediv__(self, other): 첫 번째 인수 값을 두 번째 인수 값으로 나누는 이항 연산자

$$
y = x_0 / x_1
$$
$$
{\delta y \over \delta x_0} = 1/x_1
$$
$$
{\delta y \over \delta x_1} = -x_0/(x_1)^2
$$


```python
class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y
    
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1

def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)

def rdiv(x0, x1):
    x1 = as_array(x1)
    return Div()(x1, x0)

Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv
```

### 22.4 거듭제곱

\_\_pow__: 첫 번째 인수를 두 번째 인수만큼 거듭 곱하는 이항 연산자

$$
y = x^c
$$
$$
{\delta y \over \delta x_0} = cx^{c-1}
$$

\* ${\delta y \over \delta x_c}$는 거의 사용되지 않으므로 생략

```python
class Pow(Function):
    def __init__(self, c):
        self.c = c
    
    def forward(self, x):
        y = x ** self.c
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c
        gx = c * x ** (c-1) * gy
        return gx

def pow(x, c):
    return Pow(c)(x)

Variable.__pow__ = pow
```